In [2]:
import math
from scipy.stats import norm


# norm.cdf cumulative distribution function.

def price_black_scholes(world, stock, option):
    ''' Inputs:
            world:
                acc_rfr(float): annualized_continuously_compounded_risk_free_rate
            stock:
                price(float): stock price at time 0
                annual_volatility(float): sigma
            option:
                k(float): strike price
                t(float): time to maturity of option
    '''
    r = world["acc_rfr"]
    s0, sig = stock["price"], stock["annual_volatility"]
    k, t = option["k"], option["t"]
    x = math.log( s0/(k*math.e**(-r*t)) )/(sig*math.sqrt(t)) + 0.5*sig*math.sqrt(t)

    if option["type"] == "call":
        return s0*norm.cdf(x)-k*math.e**(-r*t)*norm.cdf(x-sig*math.sqrt(t))
    else:
        return -s0*(1-norm.cdf(x))+k*math.e**(-r*t)*(1-norm.cdf(x-sig*math.sqrt(t)))

    

In [13]:
def put_call_parity(c0,world, stock,option):
    ''' Inputs 
        c0 : option premium (call or put price)
    '''
    x, t = option["k"], option["t"]
    s0 = stock["price"]
    r = world["acc_rfr"]

    if option["type"] == "call":
        return c0+x*math.e**(-r*t)-s0 #return for put
    else:
        return c0-(x*math.e**(-r*t))+s0 #return for call

In [4]:
stock={"price":60,"annual_volatility":0.3}
world={"acc_rfr":0.021} 
option={"type":"call","k":62,"t":1/4}
c0 = price_black_scholes(world,stock,option)
p0 = put_call_parity(c0,world,stock,option)
            
assert (c0,p0) == (2.860532861187565, 4.535885805382421)


In [5]:
stock={"price":60,"annual_volatility":0.3}
world={"acc_rfr":0.021} #annualized_continuously_compounded_risk_free_rate
option={"type":"call","k":62,"t":1/4}
c0 = price_black_scholes(world,stock,option)
p0 = put_call_parity( c0,world,stock,option)

            
assert (c0,p0) == (2.860532861187565, 4.535885805382421)

In [14]:
def get_price_change(world,stocks, option):

    options = [0 for _ in stocks]
    replications = [0 for _ in stocks]

    for i, stock in enumerate(stocks):
        c0 = price_black_scholes(world,stock,option)
        p0 = put_call_parity(c0,world,stock,option)
        #c_p0 = price_black_scholes(world,stock,option_replicated)
        options[i] = c0
        replications[i] = p0

    print(options,replications)
    print("option change:",options[1]-options[0])
    print("replication change:",replications[1]-replications[0])

In [15]:
stock1={"price":50,"annual_volatility":0.35}
stock2={"price":47,"annual_volatility":0.35}
world={"acc_rfr":0.0125}
option={"type":"put","k":55,"t":1/2}
get_price_change(world,[stock1,stock2],option)

[7.810250741137153, 9.730741416359109] [3.152928756850443, 2.0734194320723986]
option change: 1.9204906752219557
replication change: -1.0795093247780443


In [ ]:
stock1={"price":50,"annual_volatility":0.30}
stock2={"price":53,"annual_volatility":0.30}
world={"acc_rfr":0.031}
option={"type":"call","k":52,"t":1/2}
get_price_change(world,[stock1,stock2],option)